## Lab 0: Warm Up: Deploy Llama 2 Models on ml.inf2.24xlarge for Inference

In this lab, we'll walk you throught the process of deploying an Open Source Llama2 Model to a SageMaker endpoint for inference. We're going to leverage 1 `ml.inf2.24xlarge` machine for this and subsequent labs. In practice, you can deploy a SageMaker model behind a single load balanced endpoint with auto-scaling policies defined - allowing your LLM SaaS endpoint to scale with input demand.

<div style="background-color: #FFDDDD; border-left: 5px solid red; padding: 10px; color: black;">
    <strong>Kernel:</strong> Python 3 (ipykernel)
</div>

### _Temporary Prerequisites

In [21]:
%%bash
unzip models.zip

mv models ~/.aws/

ls -R ~/.aws

Archive:  models.zip
   creating: models/
   creating: models/sagemaker/
   creating: models/sagemaker/2017-07-24/
  inflating: models/sagemaker/2017-07-24/service-2.sdk-extras.json  
/home/sagemaker-user/.aws:
models
sagemaker

/home/sagemaker-user/.aws/models:
sagemaker

/home/sagemaker-user/.aws/models/sagemaker:
2017-07-24

/home/sagemaker-user/.aws/models/sagemaker/2017-07-24:
service-2.sdk-extras.json

/home/sagemaker-user/.aws/sagemaker:
2017-07-24

/home/sagemaker-user/.aws/sagemaker/2017-07-24:
service-2.sdk-extras.json


## Llama 2 License Agreement

In [1]:
from ipywidgets import widgets, Layout, HTML
from IPython.display import display

In [2]:
LLAMA2_EULA = False

# Creating the dropdown
dropdown = widgets.Dropdown(
    options=['True', 'False'],
    style={'description_width': 'initial'},
    layout=Layout(width='auto'),  # Adjusts the width of the dropdown to fit the content
    value='False',
    description='Accept Llama 2 EULA?',
    disabled=False,
)

# Function to be called on value change
def on_dropdown_change(change):
    global LLAMA2_EULA
    if change['type'] == 'change' and change['name'] == 'value':
        LLAMA2_EULA = eval(change['new'])
        print(f"User set EULA to {LLAMA2_EULA}")

# Observing the dropdown for changes
dropdown.observe(on_dropdown_change, names='value')

# Display the HTML link and the dropdown
display(dropdown)

Dropdown(description='Accept Llama 2 EULA?', index=1, layout=Layout(width='auto'), options=('True', 'False'), …

In [3]:
print(f"User set EULA to --> {LLAMA2_EULA}")

User set EULA to --> True


## Setup Up

Let's install some packages that would be required for this and some sub-sequent labs

In [ ]:
!python3 -m pip install ./sagemaker-2.297.1.dev0-py2.py3-none-any.whl -q

In [4]:
import os
import boto3
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [7]:
REGION = "us-west-2"

sagemaker_session = sagemaker.Session(boto_session=boto3.Session(region_name=REGION))
sm_client = boto3.client("sagemaker", region_name=REGION)
role = "arn:aws:iam::914153712152:role/workshop-studio-v2-cfn-OSE-EMR-SageMakerExecutionRole" #sagemaker.get_execution_role()

print(f"\nSageMaker python SDK version ---> {sagemaker.__version__} | Region ---> {sagemaker_session.boto_session.region_name} | Role ---> {role}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml

SageMaker python SDK version ---> 2.297.1.dev0 | Region ---> us-west-2 | Role ---> arn:aws:iam::914153712152:role/workshop-studio-v2-cfn-OSE-EMR-SageMakerExecutionRole


### Define Global Variables

In [3]:
_MODEL_NAME = "llama2"
_MODEL_SIZE = "13b"

MODEL_NAME = f"meta-{_MODEL_NAME}-{_MODEL_SIZE}-neuron-chat-tg-model"
ENDPOINT_NAME = f"meta-{_MODEL_NAME}-{_MODEL_SIZE}-neuron-chat-tg-ep"

In [5]:
MODEL_NAME, ENDPOINT_NAME

('meta-llama2-13b-neuron-chat-tg-model', 'meta-llama2-13b-neuron-chat-tg-ep')

## Let's Deploy!

![Llama 2 Model](https://venturebeat.com/wp-content/uploads/2023/07/cfr0z3n_vector_art_cybernetic_llama_wearing_sunglasses_synthwav_d3f82260-2c47-4abd-9599-b91751711f5b.png?fit=750%2C420&strip=all)

*Image Credits: https://venturebeat.com/*

### Model and Instance Configuration

In [9]:
JUMPSTART_SRC_MODEL_NAME = "meta-textgenerationneuron-llama-2-13b-f"
INSTANCE_TYPE = "ml.inf2.24xlarge"

In [10]:
# temporary workaround
os.environ.update({
    "AWS_JUMPSTART_CONTENT_BUCKET_OVERRIDE": "jumpstart-cache-alpha-us-west-2",
    "AWS_JUMPSTART_GATED_CONTENT_BUCKET_OVERRIDE": "jumpstart-private-cache-prod-us-west-2",
})

### Deploy!

<img src="https://cdn.jim-nielsen.com/ios/1024/lets-go-rocket-2018-10-15.png" width="512" height="512" />

We're going to deploy our Llama2 model on Amazon Silicon Inferentia `Inf2`. Inferentia instances are purpose built for deep learning (DL) inference. They deliver high performance at the lowest cost in Amazon EC2 for generative artificial intelligence (AI) models, including large language models (LLMs) and vision transformers. 

In [1]:
from sagemaker.jumpstart.model import JumpStartModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [12]:
llama2_13b_model = JumpStartModel(
    model_id=JUMPSTART_SRC_MODEL_NAME,
    env={
        "OPTION_DTYPE": "fp16",
        "OPTION_N_POSITIONS": "2048",
        "OPTION_TENSOR_PARALLEL_DEGREE": "12"
    },
    role=role
)

Using model 'meta-textgenerationneuron-llama-2-13b-f' with wildcard version identifier '*'. You can pin to version '1.0.0' for more stable results. Note that models may have different input/output signatures after a major version upgrade.
Using model 'meta-textgenerationneuron-llama-2-13b-f' with wildcard version identifier '*'. You can pin to version '1.0.0' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


In [13]:
%%time
print("===== SageMaker Deployment =====")

print("\nPreparing to deploy the model...")
llama2_13b_model.deploy(
    endpoint_name=ENDPOINT_NAME,
    instance_type=INSTANCE_TYPE,
    accept_eula=LLAMA2_EULA
)
print("\n===== Deployment Complete =====")

Your model is not compiled. Please compile your model before using Inferentia.


===== SageMaker Deployment =====

Preparing to deploy the model...
------------------------------------------------------!
===== Deployment Complete =====
CPU times: user 429 ms, sys: 32 ms, total: 461 ms
Wall time: 27min 40s
